In [1]:
from unstructured.partition.pdf import partition_pdf
from nltk.tokenize import sent_tokenize
import nltk
nltk.download("punkt_tab")
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Bindaas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
elements = partition_pdf("attention_is_all_you_need.pdf")
elements

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [3]:
text_list = [i.text for i in elements if i.text]
text_list[:10]

['3 2 0 2',
 'g u A 2',
 '] L C . s c [',
 '7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a',
 'Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.',
 'Attention Is All You Need',
 'Ashish Vaswani∗ Google Brain avaswani@google.com',
 'Noam Shazeer∗ Google Brain noam@google.com',
 'Niki Parmar∗ Google Research nikip@google.com',
 'Jakob Uszkoreit∗ Google Research usz@google.com']

In [4]:
sentences = []
for text in text_list:
    sentences.extend(sent_tokenize(text))
sentences[:20]

['3 2 0 2',
 'g u A 2',
 '] L C .',
 's c [',
 '7 v 2 6 7 3 0 .',
 '6 0 7 1 : v i X r a',
 'Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.',
 'Attention Is All You Need',
 'Ashish Vaswani∗ Google Brain avaswani@google.com',
 'Noam Shazeer∗ Google Brain noam@google.com',
 'Niki Parmar∗ Google Research nikip@google.com',
 'Jakob Uszkoreit∗ Google Research usz@google.com',
 'Llion Jones∗ Google Research llion@google.com',
 'Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu',
 'Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com',
 'Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com',
 'Abstract',
 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder.',
 'The best performing models also connect the encoder and decoder through an attention mechanism.',
 'We propose a n

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("paraphrase-MiniLM-L6-v2",cache_folder="./models/paraphrase-MiniLM-L6-v2")

In [6]:
embeddings = model.encode(sentences)

In [7]:
kmeans = KMeans(len(sentences)//10,random_state = 0).fit(embeddings)

In [8]:
clusters = {}
for i,label in enumerate(kmeans.labels_):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(sentences[i])
clusters

{np.int32(52): ['3 2 0 2',
  '7 v 2 6 7 3 0 .',
  '6 0 7 1 : v i X r a',
  ')V',
  ',V W V',
  '2 4 8',
  '8 1 4 16 32',
  '7 Conclusion'],
 np.int32(9): ['g u A 2',
  '1',
  '2',
  '3',
  '1√',
  '(1)',
  'See Figure 2.',
  '(2)',
  '(3)',
  '(B)',
  '(C)'],
 np.int32(17): ['] L C .',
  's c [',
  'Listing order is random.',
  'The output is computed as a weighted sum',
  'of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.',
  'The input consists of queries and keys of dimension dk, and values of dimension dv.',
  'We compute the dot products of the dk, and apply a softmax function to obtain the weights on the query with all keys, divide each by values.',
  'The keys and values are also packed together into matrices K and V .',
  'QKT √ dk',
  'variables with mean 0 and variance 1.',
  'MultiHead(Q,K,V ) = Concat(head1,...,headh)W O',
  'Where the projections are parameter matrices W Q and W O ∈ Rhdv×

In [9]:
chunks = []
for cluster in clusters.values():
    chunk = " ".join(cluster)
    chunks.append(chunk)
chunks

['3 2 0 2 7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a )V ,V W V 2 4 8 8 1 4 16 32 7 Conclusion',
 'g u A 2 1 2 3 1√ (1) See Figure 2. (2) (3) (B) (C)',
 '] L C . s c [ Listing order is random. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key. The input consists of queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the dk, and apply a softmax function to obtain the weights on the query with all keys, divide each by values. The keys and values are also packed together into matrices K and V . QKT √ dk variables with mean 0 and variance 1. MultiHead(Q,K,V ) = Concat(head1,...,headh)W O Where the projections are parameter matrices W Q and W O ∈ Rhdv×dmodel. i ∈ Rdmodel×dk, W K i ∈ Rdmodel×dk, W V i ∈ Rdmodel×dv For each of these we use dk = dv = dmodel/h = 64. Another way of describing this is as two convolutions with kernel size 1.

In [10]:
chunk_embeddings = model.encode(chunks)
chunk_embeddings

array([[ 0.25190073,  0.3876791 ,  0.15969208, ...,  0.25034207,
         0.2829401 , -0.11338047],
       [ 0.20899074,  0.3159557 , -0.15023154, ...,  0.07715788,
         0.08286655,  0.14418526],
       [-0.37988544,  0.03072838, -0.3023121 , ..., -0.05862774,
        -0.2076837 , -0.06784113],
       ...,
       [ 0.39776006, -0.17304592, -0.2986764 , ..., -0.2595495 ,
         1.4374036 ,  0.17139332],
       [ 0.01161119,  0.3880169 ,  0.69775397, ...,  0.4427333 ,
        -0.47719827,  0.19142306],
       [ 0.07544975,  0.5665533 ,  0.6950153 , ..., -0.5067977 ,
         0.04246286, -0.27757886]], shape=(62, 384), dtype=float32)

In [11]:
query = "self attention"
query_embedding = model.encode([query])
similarities = cosine_similarity(query_embedding, chunk_embeddings)[0]

top_n = 3
top_indices = np.argsort(similarities)[::-1][:top_n]

for i in top_indices:
    print(f"\n🔎 Chunk {i + 1} (Score: {similarities[i]:.4f}):\n{chunks[i]}")


🔎 Chunk 5 (Score: 0.7276):
Attention Is All You Need 3.2 Attention where headi = Attention(QW Q 3.2.3 Applications of Attention in our Model Self-Attention Recurrent Convolutional Self-Attention (restricted) 4 Why Self-Attention Motivating our use of self-attention we consider three desiderata. Effective approaches to attention- A decomposable attention Attentions here shown only for the word ‘making’. Top: Full attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5 and 6. Note that the attentions are very sharp for this word.

🔎 Chunk 22 (Score: 0.4772):
Scaled Dot-Product Attention Figure 2: (left) Scaled Dot-Product Attention. 3.2.1 Scaled Dot-Product Attention We call our particular attention "Scaled Dot-Product Attention" (Figure 2). Attention(Q,K,V ) = softmax( Dot-product attention is identical to our algorithm, except for the scaling factor . While the two are similar in theoretical complexity, dot-product attention is much faster and

In [ ]:
from huggingface_hub import login
login(token = "") # Your huggingface token here

In [13]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",cache_dir=".models/mistralai/Mistral-7B-Instruct-v0.2",quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", cache_dir=".models/mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
inputs = tokenizer("what is self attention explain with example", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


what is self attention explain with example

Self-attention is a neural network mechanism that allows a model to focus on different parts or elements of the input data while processing it. It is a type of attention mechanism that does not require an external reference or context, hence the term "self."

In traditional attention mechanisms, the model attends to specific parts of the input based on an external context or query. For example, in machine translation, the model attends to specific words or phrases in the source language based on their corresponding words or phrases in the target language.

In contrast, self-attention allows the model to attend to different parts of the input based on their relationships with each other. It does this by computing a weighted sum of the input vectors, where the weights are determined by the similarity between the input vectors.

Here's an example of how self-attention works:

Suppose we have a sequence of vectors x = [x1, x2, x3, x4], and we wa

In [24]:
# Combine the top chunks
context = "\n\n".join([chunks[i] for i in top_indices])

# Construct the prompt
prompt = f"""<s>[INST] Use the following context and your own knowledge to answer the question.

Context:
{context}

Question:
{query}

Answer: [/INST]"""

In [25]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(**inputs,max_new_tokens=1000,do_sample=True,temperature=0.7,top_p=0.9,repetition_penalty=1.1,pad_token_id=tokenizer.eos_token_id)

response = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n🧠 Model Response:\n", response)


🧠 Model Response:
 [INST] Use the following context and your own knowledge to answer the question.

Context:
Attention Is All You Need 3.2 Attention where headi = Attention(QW Q 3.2.3 Applications of Attention in our Model Self-Attention Recurrent Convolutional Self-Attention (restricted) 4 Why Self-Attention Motivating our use of self-attention we consider three desiderata. Effective approaches to attention- A decomposable attention Attentions here shown only for the word ‘making’. Top: Full attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5 and 6. Note that the attentions are very sharp for this word.

Scaled Dot-Product Attention Figure 2: (left) Scaled Dot-Product Attention. 3.2.1 Scaled Dot-Product Attention We call our particular attention "Scaled Dot-Product Attention" (Figure 2). Attention(Q,K,V ) = softmax( Dot-product attention is identical to our algorithm, except for the scaling factor . While the two are similar in theoretica